In [1]:
%cd are-gnn-defenses-robust/

[Errno 2] No such file or directory: 'are-gnn-defenses-robust/'
/home/niyati/ERSP/2-phase-attack/experiments/2-phase


/home/niyati/miniconda3/envs/ersp_v2/lib/python3.9/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [4]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *

print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [11]:
dataset_directory = "../Cora"
cora_dataset = Planetoid(root=dataset_directory, name='Cora')
data = cora_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [12]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [13]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 500)

Epoch 0, Train Loss - 3.4292547702789307, Val Loss - 5.599295616149902, Val Accuracy - 0.227
Epoch 20, Train Loss - 0.2924160659313202, Val Loss - 1.1626044511795044, Val Accuracy - 0.737
Epoch 40, Train Loss - 0.08849116414785385, Val Loss - 1.567559003829956, Val Accuracy - 0.762
Epoch 60, Train Loss - 0.04000654071569443, Val Loss - 1.8592064380645752, Val Accuracy - 0.753
Epoch 80, Train Loss - 0.06099880859255791, Val Loss - 2.0569310188293457, Val Accuracy - 0.741
Epoch 100, Train Loss - 0.040557872503995895, Val Loss - 2.2705068588256836, Val Accuracy - 0.75
Epoch 120, Train Loss - 0.03164071962237358, Val Loss - 2.2997636795043945, Val Accuracy - 0.768
Epoch 140, Train Loss - 0.03562059998512268, Val Loss - 2.4598588943481445, Val Accuracy - 0.749
Epoch 160, Train Loss - 0.03696182370185852, Val Loss - 2.232696533203125, Val Accuracy - 0.738
Epoch 180, Train Loss - 0.02980910800397396, Val Loss - 2.5133919715881348, Val Accuracy - 0.734
Epoch 200, Train Loss - 0.013250546529889

In [15]:
# Get initial accuracy
_, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")

Initial Accuracy: 0.736


In [28]:
import sys
import torch
from itertools import count
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import random
import numpy as np
import networkx as nx


# Convert the graph to NetworkX for manipulation
G = to_networkx(data, to_undirected=True)

# Store initial edge count
initial_edge_count = G.number_of_edges()

# Degree-based Edge Addition
num_edges_to_add = 100  # Define the number of edges to add

# Calculate the degree of each node
degrees = dict(G.degree())

# Group nodes by degree range (e.g., group nodes with degrees within ±1)
degree_bins = {}
for node, degree in degrees.items():
    bin_key = degree // 2  # Bin nodes by ranges of 2
    if bin_key not in degree_bins:
        degree_bins[bin_key] = []
    degree_bins[bin_key].append(node)

# Generate potential edges between nodes in the same degree bin
edges_to_add = []
for bin_nodes in degree_bins.values():
    if len(bin_nodes) > 1:
        # Randomly pair nodes within the same bin
        random.shuffle(bin_nodes)
        for i in range(len(bin_nodes)):
            for j in range(i + 1, len(bin_nodes)):
                if not G.has_edge(bin_nodes[i], bin_nodes[j]):
                    edges_to_add.append((bin_nodes[i], bin_nodes[j]))


# Randomly select a subset of edges to add
random.shuffle(edges_to_add)
edges_to_add = edges_to_add[:num_edges_to_add]


# Add the selected edges to the graph
for i, j in edges_to_add:
    G.add_edge(i, j)

# Store final edge count
final_edge_count = G.number_of_edges()

# Convert back to PyTorch Geometric format
modified_data = from_networkx(G).to(device)
modified_data.x = data.x  # Preserve original node features
modified_data.y = data.y  # Preserve original labels
modified_data.train_mask = data.train_mask
modified_data.test_mask = data.test_mask

# Evaluate the model on the modified graph
_, modified_accuracy = train.test(modified_data)
print(f"Modified Accuracy: {modified_accuracy}")

# Calculate and display the edge statistics
edge_difference = final_edge_count - initial_edge_count
print(f"Edges Added: {edge_difference}")
print(f"Percentage Increase in Edges: {100 * edge_difference / initial_edge_count:.2f}%")

# Output accuracy change
accuracy_change = modified_accuracy - initial_accuracy
print(f"Accuracy Change: {accuracy_change:.4f}")



Modified Accuracy: 0.733
Edges Added: 100
Percentage Increase in Edges: 1.89%
Accuracy Change: -0.0030


In [29]:
# Convert the graph to NetworkX for manipulation
G = to_networkx(data, to_undirected=True)

# Store initial edge count
initial_edge_count = G.number_of_edges()

# Degree-based Edge Addition
num_edges_to_add = 200  # Define the number of edges to add

# Calculate the degree of each node
degrees = dict(G.degree())

# Group nodes by degree range (e.g., group nodes with degrees within ±1)
degree_bins = {}
for node, degree in degrees.items():
    bin_key = degree // 2  # Bin nodes by ranges of 2
    if bin_key not in degree_bins:
        degree_bins[bin_key] = []
    degree_bins[bin_key].append(node)

# Generate potential edges between nodes in the same degree bin
edges_to_add = []
for bin_nodes in degree_bins.values():
    if len(bin_nodes) > 1:
        # Randomly pair nodes within the same bin
        random.shuffle(bin_nodes)
        for i in range(len(bin_nodes)):
            for j in range(i + 1, len(bin_nodes)):
                if not G.has_edge(bin_nodes[i], bin_nodes[j]):
                    edges_to_add.append((bin_nodes[i], bin_nodes[j]))


# Randomly select a subset of edges to add
random.shuffle(edges_to_add)
edges_to_add = edges_to_add[:num_edges_to_add]


# Add the selected edges to the graph
for i, j in edges_to_add:
    G.add_edge(i, j)

# Store final edge count
final_edge_count = G.number_of_edges()

# Convert back to PyTorch Geometric format
modified_data = from_networkx(G).to(device)
modified_data.x = data.x  # Preserve original node features
modified_data.y = data.y  # Preserve original labels
modified_data.train_mask = data.train_mask
modified_data.test_mask = data.test_mask

# Evaluate the model on the modified graph
_, modified_accuracy = train.test(modified_data)
print(f"Modified Accuracy: {modified_accuracy}")

# Calculate and display the edge statistics
edge_difference = final_edge_count - initial_edge_count
print(f"Edges Added: {edge_difference}")
print(f"Percentage Increase in Edges: {100 * edge_difference / initial_edge_count:.2f}%")

# Output accuracy change
accuracy_change = modified_accuracy - initial_accuracy
print(f"Accuracy Change: {accuracy_change:.4f}")

Modified Accuracy: 0.728
Edges Added: 200
Percentage Increase in Edges: 3.79%
Accuracy Change: -0.0080


In [30]:
# Convert the graph to NetworkX for manipulation
G = to_networkx(data, to_undirected=True)

# Store initial edge count
initial_edge_count = G.number_of_edges()

# Degree-based Edge Addition
num_edges_to_add = 300  # Define the number of edges to add

# Calculate the degree of each node
degrees = dict(G.degree())

# Group nodes by degree range (e.g., group nodes with degrees within ±1)
degree_bins = {}
for node, degree in degrees.items():
    bin_key = degree // 2  # Bin nodes by ranges of 2
    if bin_key not in degree_bins:
        degree_bins[bin_key] = []
    degree_bins[bin_key].append(node)

# Generate potential edges between nodes in the same degree bin
edges_to_add = []
for bin_nodes in degree_bins.values():
    if len(bin_nodes) > 1:
        # Randomly pair nodes within the same bin
        random.shuffle(bin_nodes)
        for i in range(len(bin_nodes)):
            for j in range(i + 1, len(bin_nodes)):
                if not G.has_edge(bin_nodes[i], bin_nodes[j]):
                    edges_to_add.append((bin_nodes[i], bin_nodes[j]))


# Randomly select a subset of edges to add
random.shuffle(edges_to_add)
edges_to_add = edges_to_add[:num_edges_to_add]


# Add the selected edges to the graph
for i, j in edges_to_add:
    G.add_edge(i, j)

# Store final edge count
final_edge_count = G.number_of_edges()

# Convert back to PyTorch Geometric format
modified_data = from_networkx(G).to(device)
modified_data.x = data.x  # Preserve original node features
modified_data.y = data.y  # Preserve original labels
modified_data.train_mask = data.train_mask
modified_data.test_mask = data.test_mask

# Evaluate the model on the modified graph
_, modified_accuracy = train.test(modified_data)
print(f"Modified Accuracy: {modified_accuracy}")

# Calculate and display the edge statistics
edge_difference = final_edge_count - initial_edge_count
print(f"Edges Added: {edge_difference}")
print(f"Percentage Increase in Edges: {100 * edge_difference / initial_edge_count:.2f}%")

# Output accuracy change
accuracy_change = modified_accuracy - initial_accuracy
print(f"Accuracy Change: {accuracy_change:.4f}")

Modified Accuracy: 0.733
Edges Added: 300
Percentage Increase in Edges: 5.68%
Accuracy Change: -0.0030


In [31]:
# Convert the graph to NetworkX for manipulation
G = to_networkx(data, to_undirected=True)

# Store initial edge count
initial_edge_count = G.number_of_edges()

# Degree-based Edge Addition
num_edges_to_add = 1000  # Define the number of edges to add

# Calculate the degree of each node
degrees = dict(G.degree())

# Group nodes by degree range (e.g., group nodes with degrees within ±1)
degree_bins = {}
for node, degree in degrees.items():
    bin_key = degree // 2  # Bin nodes by ranges of 2
    if bin_key not in degree_bins:
        degree_bins[bin_key] = []
    degree_bins[bin_key].append(node)

# Generate potential edges between nodes in the same degree bin
edges_to_add = []
for bin_nodes in degree_bins.values():
    if len(bin_nodes) > 1:
        # Randomly pair nodes within the same bin
        random.shuffle(bin_nodes)
        for i in range(len(bin_nodes)):
            for j in range(i + 1, len(bin_nodes)):
                if not G.has_edge(bin_nodes[i], bin_nodes[j]):
                    edges_to_add.append((bin_nodes[i], bin_nodes[j]))


# Randomly select a subset of edges to add
random.shuffle(edges_to_add)
edges_to_add = edges_to_add[:num_edges_to_add]


# Add the selected edges to the graph
for i, j in edges_to_add:
    G.add_edge(i, j)

# Store final edge count
final_edge_count = G.number_of_edges()

# Convert back to PyTorch Geometric format
modified_data = from_networkx(G).to(device)
modified_data.x = data.x  # Preserve original node features
modified_data.y = data.y  # Preserve original labels
modified_data.train_mask = data.train_mask
modified_data.test_mask = data.test_mask

# Evaluate the model on the modified graph
_, modified_accuracy = train.test(modified_data)
print(f"Modified Accuracy: {modified_accuracy}")

# Calculate and display the edge statistics
edge_difference = final_edge_count - initial_edge_count
print(f"Edges Added: {edge_difference}")
print(f"Percentage Increase in Edges: {100 * edge_difference / initial_edge_count:.2f}%")

# Output accuracy change
accuracy_change = modified_accuracy - initial_accuracy
print(f"Accuracy Change: {accuracy_change:.4f}")

Modified Accuracy: 0.729
Edges Added: 1000
Percentage Increase in Edges: 18.95%
Accuracy Change: -0.0070


In [32]:
# Convert the graph to NetworkX for manipulation
G = to_networkx(data, to_undirected=True)

# Store initial edge count
initial_edge_count = G.number_of_edges()

# Degree-based Edge Addition
num_edges_to_add = 2000  # Define the number of edges to add

# Calculate the degree of each node
degrees = dict(G.degree())

# Group nodes by degree range (e.g., group nodes with degrees within ±1)
degree_bins = {}
for node, degree in degrees.items():
    bin_key = degree // 2  # Bin nodes by ranges of 2
    if bin_key not in degree_bins:
        degree_bins[bin_key] = []
    degree_bins[bin_key].append(node)

# Generate potential edges between nodes in the same degree bin
edges_to_add = []
for bin_nodes in degree_bins.values():
    if len(bin_nodes) > 1:
        # Randomly pair nodes within the same bin
        random.shuffle(bin_nodes)
        for i in range(len(bin_nodes)):
            for j in range(i + 1, len(bin_nodes)):
                if not G.has_edge(bin_nodes[i], bin_nodes[j]):
                    edges_to_add.append((bin_nodes[i], bin_nodes[j]))


# Randomly select a subset of edges to add
random.shuffle(edges_to_add)
edges_to_add = edges_to_add[:num_edges_to_add]


# Add the selected edges to the graph
for i, j in edges_to_add:
    G.add_edge(i, j)

# Store final edge count
final_edge_count = G.number_of_edges()

# Convert back to PyTorch Geometric format
modified_data = from_networkx(G).to(device)
modified_data.x = data.x  # Preserve original node features
modified_data.y = data.y  # Preserve original labels
modified_data.train_mask = data.train_mask
modified_data.test_mask = data.test_mask

# Evaluate the model on the modified graph
_, modified_accuracy = train.test(modified_data)
print(f"Modified Accuracy: {modified_accuracy}")

# Calculate and display the edge statistics
edge_difference = final_edge_count - initial_edge_count
print(f"Edges Added: {edge_difference}")
print(f"Percentage Increase in Edges: {100 * edge_difference / initial_edge_count:.2f}%")

# Output accuracy change
accuracy_change = modified_accuracy - initial_accuracy
print(f"Accuracy Change: {accuracy_change:.4f}")

Modified Accuracy: 0.688
Edges Added: 2000
Percentage Increase in Edges: 37.89%
Accuracy Change: -0.0480
